<a href="https://colab.research.google.com/github/akshayjain777/AI/blob/master/ResNet_practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F ## activation function
import torchvision ##contains dataset
import pdb

class baseBlock(torch.nn.Module):
  expansion=1
  def __init__(self,input_planes,planes,stride=1,dim_change=None):
    super(baseBlock, self).__init__()
    #declare convolution layers with batch norms
    self.conv1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
    self.bn1= torch.nn.BatchNorm2d(planes)
    self.conv2 = torch.nn.Conv2d(planes,planes,stride=1,kernel_size=3,padding=1)
    self.bn2=torch.nn.BatchNorm2d(planes)
    self.dim_change=dim_change
  def forward(self,x):
    #save the residue
    res = x
    output=F.relu(self.bn1(self.conv1(x)))
    output=self.bn2(self.conv2(output))

    if self.dim_change is not None:
      res=self.dim_change(res)

    output += res
    output= F.relu(output)

    return output

class bottleNeck(torch.nn.Module):
  expansion=4
  def __init__(self,input_planes,planes,stride=1,dim_change=None):
    super(bottleNeck,self).__init__()
    self.conv1 = torch.nn.Conv2d(input_planes,planes,kernel_size=1,stride=1)
    self.bn1=torch.nn.BatchNorm2d(planes)
    self.conv2 = torch.nn.Conv2d(planes,planes,kernel_size=3,stride=stride,padding=1)
    self.bn2= torch.nn.BatchNorm2d(planes)
    self.conv3= torch.nn.Conv2d(planes,planes*self.expansion,kernel_size=1)
    self.dim_change=dim_change
  
  def forward(self,x):
    res=x
    output=F.relu(self.bn1(self.conv1(x)))
    output=F.relu(self.bn2(self.conv2(output)))
    output= self.bn3(self.conv3(output))

    if self.dim_change is not None:
      res=self.dim_change(res)

    output += res
    output= F.relu(output)

    return output

class ResNet(torch.nn.Module):
  def __init__(self,block,num_layers,classes=10):
    super(ResNet,self).__init__()
    #according to research paper
    self.input_planes=64
    self.conv1= torch.nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
    self.bn1= torch.nn.BatchNorm2d(64)
    self.layer1 = self._layer(block,64,num_layers[0],stride=1)
    self.layer2 =  self._layer(block,128,num_layers[1],stride=2)
    self.layer3 = self._layer(block,256,num_layers[2],stride=2)
    self.layers4 = self._layer(block,512,num_layers[3],stride=2)
    self.averagePool = torch.nn.AvgPool2d(kernel_size=4,stride=1)
    self.fc=torch.nn.Linear(512*block.expansion,classes)

  def _layer(self,block,planes,num_layers,stride=1):
    dim_change=None
    if stride!=1 or planes !=self.input_planes*block.expansion:
      dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes*block.expansion,kernel_xize=1,stride=stride),torch.nn.BatchNorm2d(planes*block.expansion))
    
    netLayers= []
    netlayers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
    self.input_palnes= planes * block.expansion
    for i in range(1,num_layers):
      netLayers.append(block(self.input_planes,planes))
      self.input_planes= planes * block.expansion
    
    return torch.nn.Sequential(*netLayers)
  
  def forward(self,x):
    x= F.relu(self.bn1(self.conv1(x)))

    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)

    x=F.avg_pool2d(x,4)
    x=x.view(x.size(0),-1)
    x=self.fc(x)

    return x
  def test():
    #to convert data from PIL to tensor

    transform= transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
    )

    #Load train and test

    train=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
    trainset= torch.utils.data.DataLoader(train,batch_size=128,shuffle=True)

    test=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
    testset= torch.utils.data.DataLoader(test,batch_size=128,shuffle=True)

    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)


  


In [2]:
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

cpu


In [3]:
#ResNet 50
net=ResNet(bottleNeck,[3,4,6,3]) 
net.to(device)
costFunc= torch.nnCrossEntropyLoss()
optimizer= torch.optim.SGD(net.parameters(),lr=0.02,momentum=0.9)

for i in range(100):
  class=0
  for i,batch in enumerate(trainset,0):
    data.output = batch
    data.output=data.to(device),output.to(device)
    prediction = net(data)

    loss=costFunc(prediction,output)
    class=loss.item()

    optimizer.zero_grad()
    loss.backwad()
    optimizer.step()

    if i%100 ==0:
      print('[%d %d]%'(epoch+1,i+1,class/1000))
      class=0


SyntaxError: ignored

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F ## activation function
import torchvision ##contains dataset
import pdb

torch.randn(1, 3, 32, 32)

tensor([[[[ 1.1818e+00,  1.2564e-01, -7.1512e-01,  ..., -2.0236e+00,
            1.0994e+00,  1.5270e-01],
          [-3.9848e-01, -1.9114e+00,  5.1754e-01,  ..., -1.0271e+00,
           -1.2799e+00,  4.4580e-01],
          [ 2.0552e+00,  5.8987e-01, -1.2892e-01,  ...,  2.0183e+00,
           -8.0929e-01, -1.8844e+00],
          ...,
          [ 8.4641e-01, -3.2796e-01,  7.0259e-01,  ..., -1.3281e+00,
           -1.2581e+00, -3.2875e-01],
          [-4.5183e-01,  1.8537e-02,  4.5907e-01,  ..., -1.2881e+00,
           -7.7791e-01, -2.8893e-01],
          [-2.6514e-03,  1.0182e-01, -4.6682e-01,  ..., -1.1092e+00,
            7.1665e-01, -2.3300e-01]],

         [[-1.0246e+00, -5.5837e-01, -1.2189e+00,  ..., -4.9854e-01,
            1.7785e+00,  4.4672e-01],
          [ 5.0185e-01,  5.7404e-01, -8.8522e-01,  ..., -1.0465e-01,
           -2.0561e+00,  1.9382e-01],
          [ 1.4283e+00,  6.3552e-01,  1.3217e+00,  ...,  1.2668e+00,
            1.0689e+00,  3.9451e-01],
          ...,
     